In [ ]:
import requests
import os

TENANT_ID = os.environ.get("AZURE_TENANT_ID")
CLIENT_ID = os.environ.get("AZURE_CLIENT_ID")
CLIENT_SECRET = os.environ.get("AZURE_CLIENT_SECRET")

TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

data = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "scope": "https://graph.microsoft.com/.default",
}

response = requests.post(TOKEN_URL, data=data)
access_token = response.json().get("access_token")
print(access_token)


In [2]:
import msal
import requests
TENANT_ID = "10b6e800-771a-4085-b222-3285c882edcc"
CLIENT_ID = "bf5e735e-ec32-4fab-a7a5-75b30b0b99bb"
# === Azure App Info ===
client_id = "bf5e735e-ec32-4fab-a7a5-75b30b0b99bb"
tenant_id = "10b6e800-771a-4085-b222-3285c882edcc"
authority_url = f'https://login.microsoftonline.com/{tenant_id}'
scope = ['https://analysis.windows.net/powerbi/api/.default']
redirect_uri = 'http://localhost:8000'

# === Token Acquisition via Device Code Flow ===
app = msal.PublicClientApplication(client_id=client_id, authority=authority_url)

# This will prompt the user to authenticate via browser
flow = app.initiate_device_flow(scopes=scope)
if "user_code" not in flow:
    raise Exception("Failed to create device flow")

print(f"Please go to {flow['verification_uri']} and enter code: {flow['user_code']}")

# Wait for user to authenticate
result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result['access_token']
    headers = {'Authorization': f'Bearer {access_token}'}

    # Example: List Power BI Groups (Workspaces)
    response = requests.get('https://api.powerbi.com/v1.0/myorg/groups', headers=headers)
    print(response.json())
else:
    print("Error acquiring token:")
    print(result.get("error_description"))

Please go to https://microsoft.com/devicelogin and enter code: LSCRSDJV4
{'@odata.context': 'https://api.powerbi.com/v1.0/myorg/$metadata#groups', '@odata.count': 5, 'value': [{'id': '6e1eac53-fede-4231-b58a-d6afd0272a77', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'HalloWelt'}, {'id': 'ac2699a2-07ce-461b-a74a-7a3033a6a6d8', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'tesgg'}, {'id': '1453dbc1-18ca-4a23-bc06-ef7689f04320', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'Tutorial_Data'}, {'id': 'e64c0866-be91-4eac-a5b7-3ade651e8563', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'SA_20230520'}, {'id': '9b2ff77a-7244-4540-ab24-d8bf46d6636e', 'isReadOnly': False, 'isOnDedicatedCapacity': False, 'type': 'Workspace', 'name': 'PBI_test_WS'}]}


In [3]:
group_url = "https://api.powerbi.com/v1.0/myorg/groups"
groups = requests.get(group_url, headers=headers).json()

# Get reports from each group
for group in groups.get("value", []):
    print(f"\n📁 Workspace: {group['name']}")
    print(f"\n📁 Workspace: {group['id']}")
    report_url = f"https://api.powerbi.com/v1.0/myorg/groups/{group['id']}/reports"
    reports = requests.get(report_url, headers=headers).json()
    for report in reports.get("value", []):
        print(f"  📄 Report: {report['name']} (ID: {report['id']})")


📁 Workspace: HalloWelt

📁 Workspace: 6e1eac53-fede-4231-b58a-d6afd0272a77
  📄 Report: publishBI (ID: 04893380-eb8a-4f6c-9778-2da8a4b1c2be)

📁 Workspace: tesgg

📁 Workspace: ac2699a2-07ce-461b-a74a-7a3033a6a6d8

📁 Workspace: Tutorial_Data

📁 Workspace: 1453dbc1-18ca-4a23-bc06-ef7689f04320
  📄 Report: First_MSTR_Power_BI (ID: a12b2390-2875-43ba-b30b-c301e0cb416c)
  📄 Report: publishBI (ID: eaa28cc9-0fef-4ba0-830a-716aac27a333)
  📄 Report: Powered by MSTR BI (ID: a9b99a77-83de-45f7-b770-350118f63c98)

📁 Workspace: SA_20230520

📁 Workspace: e64c0866-be91-4eac-a5b7-3ade651e8563
  📄 Report: publishBI (ID: 8a3a4eeb-d7e2-42f9-aca4-56bf508a60c5)

📁 Workspace: PBI_test_WS

📁 Workspace: 9b2ff77a-7244-4540-ab24-d8bf46d6636e
  📄 Report: Sales Analysis - Enforce model security (ID: 4ee88dc8-ef05-4a85-a8e9-f61c27920704)
  📄 Report: Sales Analysis (ID: 052bf2cf-5d11-4368-90a3-5961575d3568)
  📄 Report: Adventure Works DW 2020 M02 (ID: aa79c037-ee45-4c1e-8f2b-e33b30aa64e6)
  📄 Report: Report Usage Metr

In [11]:
workspace_id="1453dbc1-18ca-4a23-bc06-ef7689f04320"
report_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports"
reports = requests.get(report_url, headers=headers).json()
reports

{'@odata.context': 'https://api.powerbi.com/v1.0/myorg/groups/1453dbc1-18ca-4a23-bc06-ef7689f04320/$metadata#reports',
 'value': [{'id': 'a12b2390-2875-43ba-b30b-c301e0cb416c',
   'reportType': 'PowerBIReport',
   'name': 'First_MSTR_Power_BI',
   'webUrl': 'https://app.powerbi.com/groups/1453dbc1-18ca-4a23-bc06-ef7689f04320/reports/a12b2390-2875-43ba-b30b-c301e0cb416c',
   'embedUrl': 'https://app.powerbi.com/reportEmbed?reportId=a12b2390-2875-43ba-b30b-c301e0cb416c&groupId=1453dbc1-18ca-4a23-bc06-ef7689f04320&w=2&config=eyJjbHVzdGVyVXJsIjoiaHR0cHM6Ly9XQUJJLUdFUk1BTlktV0VTVC1DRU5UUkFMLVBSSU1BUlktcmVkaXJlY3QuYW5hbHlzaXMud2luZG93cy5uZXQiLCJlbWJlZEZlYXR1cmVzIjp7InVzYWdlTWV0cmljc1ZOZXh0Ijp0cnVlfX0%3d',
   'isFromPbix': False,
   'isOwnedByMe': True,
   'datasetId': '2d3de4b6-025a-4ba8-9e6b-838bde995235',
   'datasetWorkspaceId': '0c8ce916-6acd-4b87-804a-b8aeb59c3674',
   'users': [],
   'subscriptions': [],
   'reportFlags': 0},
  {'id': 'eaa28cc9-0fef-4ba0-830a-716aac27a333',
   'reportT

In [12]:
reportId="a9b99a77-83de-45f7-b770-350118f63c98"
report_url = f"https://api.powerbi.com/v1.0/myorg/reports/{reportId}/pages"
reports = requests.get(report_url, headers=headers).json()
reports

{'@odata.context': 'https://api.powerbi.com/v1.0/myorg/$metadata#pages',
 'value': [{'name': 'ReportSection', 'displayName': 'Page 1', 'order': 0},
  {'name': 'ReportSectionbe0cdcb5ab4533b6e402',
   'displayName': 'DrillThrough',
   'order': 1},
  {'name': 'ReportSection5e47f6044a00d9e940ac',
   'displayName': 'ToolTip',
   'order': 2}]}

In [2]:

reportId="a9b99a77-83de-45f7-b770-350118f63c98"
pageName="ReportSectionbe0cdcb5ab4533b6e402"
page_url=f"https://api.powerbi.com/v1.0/myorg/reports/{reportId}/pages/{pageName}"
page_cont = requests.get(page_url, headers=headers).json()
page_cont

{'@odata.context': 'https://api.powerbi.com/v1.0/myorg/$metadata#pages/$entity',
 'name': 'ReportSectionbe0cdcb5ab4533b6e402',
 'displayName': 'DrillThrough',
 'order': 1}

In [2]:
import zipfile
import json
file_j="D:/shared_drive/Power BI/Powered by MSTR BI_pbib.pbib"
with zipfile.ZipFile(file_j, 'r') as z:
    with z.open("report.json") as f:
        report = json.load(f)

print(report.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'D:/shared_drive/Power BI/Powered by MSTR BI_pbib.pbib'